## 0. Required Libraries

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error

import joblib
import json
import pandas as pd
import numpy as np
import copy
import hashlib

import src.util as util

## 1. Load Configuration File

In [ ]:
params = util.load_config()

## 2. Load Dataset

In [ ]:
def load_train_feng(params: dict) -> pd.DataFrame:
    # Load train set
    x_train = util.pickle_load(params["train_feng_set_path"][0])
    y_train = util.pickle_load(params["train_feng_set_path"][1])

    return x_train, y_train

def load_valid_feng(params: dict) -> pd.DataFrame:
    # Load valid set
    x_valid = util.pickle_load(params["valid_feng_set_path"][0])
    y_valid = util.pickle_load(params["valid_feng_set_path"][1])

    return x_valid, y_valid

def load_test_feng(params: dict) -> pd.DataFrame:
    # Load test set
    x_test = util.pickle_load(params["test_feng_set_path"][0])
    y_test = util.pickle_load(params["test_feng_set_path"][1])

    return x_test, y_test

In [ ]:
def load_dataset(params: dict) -> pd.DataFrame:
    # Debug message
    util.print_debug("Loading dataset.")

    # Load train set
    x_train, y_train = load_train_feng(params)

    # Load valid set
    x_valid, y_valid = load_valid_feng(params)

    # Load test set
    x_test, y_test = load_test_feng(params)

    # Debug message
    util.print_debug("Dataset loaded.")

    # Return the dataset
    return x_train, y_train, x_valid, y_valid, x_test, y_test

## 3. Create Training Log Template

In [ ]:
def training_log_template() -> dict:
    # Debug message
    util.print_debug("Creating training log template.")
    
    # Template of training log
    logger = {
        "model_name" : [],
        "model_uid" : [],
        "training_time" : [],
        "training_date" : [],
        "performance" : [],
        "mse" : [],
        "data_configurations" : [],
    }

    # Debug message
    util.print_debug("Training log template created.")

    # Return training log template
    return logger

In [ ]:
def training_log_updater(current_log: dict, params: dict) -> list:
    # Create copy of current log
    current_log = copy.deepcopy(current_log)

    # Path for training log file
    log_path = params["training_log_path"]

    # Try to load training log file
    try:
        with open(log_path, "r") as file:
            last_log = json.load(file)
        file.close()

    # If file not found, create a new one
    except FileNotFoundError as fe:
        with open(log_path, "w") as file:
            file.write("[]")
        file.close()

        with open(log_path, "r") as file:
            last_log = json.load(file)
        file.close()
    
    # Add current log to previous log
    last_log.append(current_log)

    # Save updated log
    with open(log_path, "w") as file:
        json.dump(last_log, file)
        file.close()

    # Return log
    return last_log

## 4. Training and Evaluation

### 4.1 Create Model Object

In [ ]:
def create_model_object(params: dict) -> list:
    # Debug message
    util.print_debug("Creating model objects.")

    # Create model objects

    lnr = LinearRegression()
    rdg = Ridge()
    dct = DecisionTreeRegressor()
    ran = RandomForestRegressor()
    knn = KNeighborsRegressor()
    xgb = XGBRegressor()

    # Create list of model
    list_of_model = [
        { "model_name": lnr.__class__.__name__, "model_object": lnr, "model_uid": ""},
        { "model_name": rdg.__class__.__name__, "model_object": rdg, "model_uid": ""},
        { "model_name": dct.__class__.__name__, "model_object": dct, "model_uid": ""},
        { "model_name": ran.__class__.__name__, "model_object": ran, "model_uid": ""},
        { "model_name": knn.__class__.__name__, "model_object": knn, "model_uid": ""},
        { "model_name": xgb.__class__.__name__, "model_object": xgb, "model_uid": ""}
    ]

    # Debug message
    util.print_debug("Model objects created.")

    # Return the list of model
    return list_of_model

### 4.2 Training Baseline Model

In [ ]:
def train_eval(configuration_model: str, params: dict, hyperparams_model: list = None):
    # Load dataset
    x_train, y_train, \
    x_valid, y_valid, \
    x_test, y_test = load_dataset(params)

    # Variabel to store trained models
    list_of_trained_model = dict()

    # Create log template
    training_log = training_log_template()

    # Training for every data configuration
    for config_data in x_train.columns:
        # Debug message
        util.print_debug("Training model based on configuration data: {}".format(config_data))

        # Create model objects
        if hyperparams_model == None:
            list_of_model = create_model_object(params)
        else:
            list_of_model = copy.deepcopy(hyperparams_model)

        # Variabel to store trained model
        trained_model = list()

        # Load train data based on its configuration
        x_train_data = x_train[[config_data]]
        y_train_data = y_train

        # Train each model by current dataset configuration
        for model in list_of_model:
            # Debug message
            util.print_debug("Training model: {}".format(model["model_name"]))

            # Training
            training_time = util.time_stamp()
            model["model_object"].fit(x_train_data, y_train_data)
            training_time = (util.time_stamp() - training_time).total_seconds()

            # Debug message
            util.print_debug("Evalutaing model: {}".format(model["model_name"]))

            # Evaluation
            y_predict = model["model_object"].predict(x_valid[[config_data]])
            performance = mean_squared_error(y_valid, y_predict)

            # Debug message
            util.print_debug("Logging: {}".format(model["model_name"]))

            # Create UID
            uid = hashlib.md5(str(training_time).encode()).hexdigest()

            # Assign model's UID
            model["model_uid"] = uid

            # Create training log data
            training_log["model_name"].append("{}-{}".format(configuration_model, model["model_name"]))
            training_log["model_uid"].append(uid)
            training_log["training_time"].append(training_time)
            training_log["training_date"].append(util.time_stamp())
            training_log["performance"].append(performance)
            training_log["mse"].append(performance)
            training_log["data_configurations"].append('no configuration')

            # Collect current trained model
            trained_model.append(copy.deepcopy(model))

            # Debug message
            util.print_debug("Model {} has been trained for configuration data {}.".format(model["model_name"], config_data))
        
        # Collect current trained list of model
        list_of_trained_model[config_data] = copy.deepcopy(trained_model)
    
    # Debug message
    util.print_debug("All combination models and configuration data has been trained.")
    
    # Return list trained model
    return list_of_trained_model, training_log

In [ ]:
list_of_trained_model, training_log = train_eval("Baseline", params)

### 4.3 Choose Best Performance Baseline Model

In [ ]:
def get_production_model(list_of_model, training_log, params):
    # Create copy list of model
    list_of_model = copy.deepcopy(list_of_model)
    
    # Debug message
    util.print_debug("Choosing model by metrics score.")

    # Create required predefined variabel
    curr_production_model = None
    prev_production_model = None
    production_model_log = None

    # Debug message
    util.print_debug("Converting training log type of data from dict to dataframe.")

    # Convert dictionary to pandas for easy operation
    training_log = pd.DataFrame(copy.deepcopy(training_log))

    # Debug message
    util.print_debug("Trying to load previous production model.")

    # Check if there is a previous production model
    try:
        prev_production_model = util.pickle_load(params["production_model_path"])
        util.print_debug("Previous production model loaded.")

    except FileNotFoundError as fe:
        util.print_debug("No previous production model detected, choosing best model only from current trained model.")

    # If previous production model detected:
    if prev_production_model != None:
        # Debug message
        util.print_debug("Loading validation data.")
        x_valid, y_valid = load_valid_feng(params)
        
        # Debug message
        util.print_debug("Checking compatibilty previous production model's input with current train data's features.")

        # Check list features of previous production model and current dataset
        production_model_features = set(prev_production_model["model_data"]["model_object"].feature_names_in_)
        current_dataset_features = set(x_valid.columns)
        number_of_different_features = len((production_model_features - current_dataset_features) | (current_dataset_features - production_model_features))

        # If feature matched:
        if number_of_different_features == 0:
            # Debug message
            util.print_debug("Features compatible.")

            # Debug message
            util.print_debug("Reassesing previous model performance using current validation data.")

            # Re-predict previous production model to provide valid metrics compared to other current models
            y_pred = prev_production_model["model_data"]["model_object"].predict(x_valid)

            # Re-asses prediction result
            eval_res = mean_squared_error(y_valid, y_pred)

            # Debug message
            util.print_debug("Assessing complete.")

            # Debug message
            util.print_debug("Storing new metrics data to previous model structure.")

            # Update their performance log
            prev_production_model["model_log"]["performance"] = eval_res
            prev_production_model["model_log"]["mse"] = eval_res["mse"]

            # Debug message
            util.print_debug("Adding previous model data to current training log and list of model")

            # Added previous production model log to current logs to compere who has the greatest f1 score
            training_log = pd.concat([training_log, pd.DataFrame([prev_production_model["model_log"]])])

            # Added previous production model to current list of models to choose from if it has the greatest f1 score
            list_of_model["prev_production_model"] = [copy.deepcopy(prev_production_model["model_data"])]
        else:
            # To indicate that we are not using previous production model
            prev_production_model = None

            # Debug message
            util.print_debug("Different features between production model with current dataset is detected, ignoring production dataset.")

    # Debug message
    util.print_debug("Sorting training log by mse and training time.")

    # Sort training log by mse and training time
    best_model_log = training_log.sort_values(["mse", "training_time"], ascending = [False, True]).iloc[0]
    
    # Debug message
    util.print_debug("Searching model data based on sorted training log.")

    # Get model object with greatest mse score macro avg by using UID
    for configuration_data in list_of_model:
        for model_data in list_of_model[configuration_data]:
            if model_data["model_uid"] == best_model_log["model_uid"]:
                curr_production_model = dict()
                curr_production_model["model_data"] = copy.deepcopy(model_data)
                curr_production_model["model_log"] = copy.deepcopy(best_model_log.to_dict())
                curr_production_model["model_log"]["model_name"] = "Production-{}".format(curr_production_model["model_data"]["model_name"])
                curr_production_model["model_log"]["training_date"] = str(curr_production_model["model_log"]["training_date"])
                production_model_log = training_log_updater(curr_production_model["model_log"], params)
                break
    
    # In case UID not found
    if curr_production_model == None:
        raise RuntimeError("The best model not found in your list of model.")
    
    # Debug message
    util.print_debug("Model chosen.")

    # Dump chosen production model
    util.pickle_dump(curr_production_model, params["production_model_path"])
    
    # Return current chosen production model, log of production models and current training log
    return curr_production_model, production_model_log, training_log
    

In [ ]:
model, production_model_log, training_logs = get_production_model(list_of_trained_model, training_log, params)

## 5. Hyperparameter Tuning

In [29]:
def create_grid_params(model_name: str) -> dict:

    # Define models parameters
    grid_params_xgb = {
        "n_estimators": [50, 100, 200, 300, 400, 500]
    }
    grid_params_dct = {
        "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
        "min_samples_split": [1, 2, 4, 6, 10, 15, 20, 25],
        "min_samples_leaf": [1, 2, 4, 6, 10, 15, 20, 25]
    }
    grid_params_knn = {
        "weights": ["uniform", "distance"],
        "algorithm": ["ball_tree", "kd_tree", "brute"],
        "n_neighbors": [2, 3, 4, 5, 6, 10, 15, 20, 25],
        "leaf_size": [2, 3, 4, 5, 6, 10, 15, 20, 25],
    }
    grid_params_ridge = {
        "alpha": [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.6, 0.9, 1],
        "solver": ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"],
    }
    grid_params_rfr = {
        "n_estimators": [50, 100, 200, 300, 400, 500],
        "criterion": ["mse", "mae"],
        "min_samples_split": [1, 2, 4, 6, 10, 15, 20, 25],
        "min_samples_leaf": [1, 2, 4, 6, 10, 15, 20, 25]
    }

    # Combine all models parameters into one
    grid_params = {
        "XGBRegressor": grid_params_xgb,
        "DecisionTreeRegressor": grid_params_dct,
        "KNeighborsRegressor": grid_params_knn,
        "Ridge": grid_params_ridge,
        "RandomForestRegressor": grid_params_rfr
    }

    # Return distribution of model parameters
    return grid_params[model_name]

In [30]:
from sklearn.model_selection import GridSearchCV

def hyper_params_tuning(model: dict) -> list:
    # Create copy of current best baseline model
    model = copy.deepcopy(model)

    # Create model's parameter distribution
    grid_params = create_grid_params(model["model_data"]["model_name"])

    # Create model object
    model_gsc = GridSearchCV(model["model_data"]["model_object"], grid_params, n_jobs = -1)
    model_data = {
        "model_name": model["model_data"]["model_name"],
        "model_object": model_gsc,
        "model_uid": ""
    }
    
    # Return model object
    return [model_data]



In [31]:
list_of_trained_model, training_log = train_eval("Hyperparams_Tuning", params, hyper_params_tuning(model))

2023-04-12 08:08:18.744462 Loading dataset.
2023-04-12 08:08:18.749864 Dataset loaded.
2023-04-12 08:08:18.749886 Creating training log template.
2023-04-12 08:08:18.749891 Training log template created.
2023-04-12 08:08:18.749944 Training model based on configuration data: INTD.JK
2023-04-12 08:08:18.751516 Training model: DecisionTreeRegressor


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pac

2023-04-12 08:08:20.148693 Evalutaing model: DecisionTreeRegressor
2023-04-12 08:08:20.149575 Logging: DecisionTreeRegressor
2023-04-12 08:08:20.150204 Model DecisionTreeRegressor has been trained for configuration data INTD.JK.
2023-04-12 08:08:20.150673 Training model based on configuration data: ULTJ.JK
2023-04-12 08:08:20.150943 Training model: DecisionTreeRegressor


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pac

2023-04-12 08:08:20.448915 Evalutaing model: DecisionTreeRegressor
2023-04-12 08:08:20.449751 Logging: DecisionTreeRegressor
2023-04-12 08:08:20.450280 Model DecisionTreeRegressor has been trained for configuration data ULTJ.JK.
2023-04-12 08:08:20.450759 Training model based on configuration data: PDES.JK
2023-04-12 08:08:20.450995 Training model: DecisionTreeRegressor


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pac

2023-04-12 08:08:21.985497 Evalutaing model: DecisionTreeRegressor
2023-04-12 08:08:21.986111 Logging: DecisionTreeRegressor
2023-04-12 08:08:21.986754 Model DecisionTreeRegressor has been trained for configuration data APII.JK.
2023-04-12 08:08:21.987216 Training model based on configuration data: TLKM.JK
2023-04-12 08:08:21.987457 Training model: DecisionTreeRegressor


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pac

2023-04-12 08:08:22.252810 Evalutaing model: DecisionTreeRegressor
2023-04-12 08:08:22.253410 Logging: DecisionTreeRegressor
2023-04-12 08:08:22.254052 Model DecisionTreeRegressor has been trained for configuration data TLKM.JK.
2023-04-12 08:08:22.254519 Training model based on configuration data: JKON.JK
2023-04-12 08:08:22.254764 Training model: DecisionTreeRegressor
2023-04-12 08:08:22.542141 Evalutaing model: DecisionTreeRegressor
2023-04-12 08:08:22.542749 Logging: DecisionTreeRegressor
2023-04-12 08:08:22.543368 Model DecisionTreeRegressor has been trained for configuration data JKON.JK.
2023-04-12 08:08:22.543822 All combination models and configuration data has been trained.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-pac

## 6. Take a Look at MSE & Prediction Plot

In [ ]:
import matplotlib.pyplot as plt

x_valid, y_valid = load_valid_feng(params)
y_pred = model["model_data"]["model_object"].predict(x_valid)

# Calculate Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error:", mse)

# Scatter plot of actual vs. predicted values
plt.scatter(y_valid, y_pred, alpha=0.5)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values")

# Add a diagonal line representing a perfect prediction
plt.plot([min(y_valid), max(y_valid)], [min(y_valid), max(y_valid)], color='red', linestyle='--')

plt.show()


In [ ]:
x_valid, y_valid = load_valid_feng(params)

x_train, y_train = load_train_feng(params)

# After loading training data
print("Training data columns:", x_train.columns)

# After loading validation data
print("Validation data columns:", x_valid.columns)


In [ ]:
# After loading training data
print("Training data shape:", x_train.shape)

# After loading validation data
print("Validation data shape:", x_valid.shape)